# Tutorial 3: Fine-Tuning a Cohere LLM with Medical Data

• Find the dataset preparation  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2010%20-%20Create_Dataset_For_Cohere_Fine_Tuning.ipynb)  and the fine-tuning  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2010%20-%20Fine_Tuning_using_Cohere_for_Medical_Data.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

Using a proprietary model simplifies the fine-tuning process by simply supplying sample inputs and outputs, with the platform managing the actual fine-tuning. For example, in a classification model, a typical input would be a pair of <text, label>.

Cohere offers a range of specialized models for specific use cases and different functions, including  [rerank](https://txt.cohere.com/rerank/),  [embedding](https://docs.cohere.com/docs/multilingual-language-models), and  [chat](https://cohere.com/chat), all accessible via APIs. Users can create  [custom models](https://txt.cohere.com/custom-command-models/)  for three primary objectives: 1) Generative tasks where the model produces text output, 2) Classification tasks where the model categorizes text, and 3) Rerank tasks to improve semantic search results.

In this tutorial, we are fine-tuning a proprietary LLM developed by  [Cohere](https://cohere.com/)  for medical text analysis to perform  [Named Entity Recognition (NER)](https://en.wikipedia.org/wiki/Named-entity_recognition). NER enables models to recognize multiple entities in text, such as names, locations, and dates. We will fine-tune a model to extract information about diseases, substances, and their interactions from medical paper abstracts.

## Cohere API

The Cohere platform provides a selection of base models designed for different purposes. You can choose between base models with quicker performance or command models with more advanced capabilities for generative tasks. Each type also has a “light” version for additional flexibility.

[Create an account](https://dashboard.cohere.com/welcome/register)  on their platform to use the Cohere API at dashboard.cohere.com. Navigate to the “API Keys” section to obtain a Trial key, which allows free usage with certain rate limitations. This key is not for production environments but offers an excellent opportunity to experiment with the models before using them for production.

Install the Cohere Python SDK to access their API:

pip install cohere

Build a Cohere object with your API key and a prompt to generate a response to your request. You can use the code below but change the API placeholder with your key:

    pip install cohere

Build a Cohere object with your API key and a prompt to generate a response to your request. You can use the code below but change the API placeholder with your key:

In [ ]:
import json
from fine_tuning_custom_utils.helper import get_cohere_api_key
COHERE_API_KEY = get_cohere_api_key()

In [ ]:
import cohere  

co = cohere.Client("<API_KEY>")

prompt = """The following article contains technical terms including diseases, drugs and chemicals. Create a list only of the diseases mentioned.

Progressive neurodegeneration of the optic nerve and the loss of retinal ganglion cells is a hallmark of glaucoma, the leading cause of irreversible blindness worldwide, with primary open-angle glaucoma (POAG) being the most frequent form of glaucoma in the Western world. While some genetic mutations have been identified for some glaucomas, those associated with POAG are limited and for most POAG patients, the etiology is still unclear. Unfortunately, treatment of this neurodegenerative disease and other retinal degenerative diseases is lacking. For POAG, most of the treatments focus on reducing aqueous humor formation, enhancing uveoscleral or conventional outflow, or lowering intraocular pressure through surgical means. These efforts, in some cases, do not always lead to a prevention of vision loss and therefore other strategies are needed to reduce or reverse the progressive neurodegeneration. In this review, we will highlight some of the ocular pharmacological approaches that are being tested to reduce neurodegeneration and provide some form of neuroprotection.

List of extracted diseases:"""

response = co.generate(  
    model='command',  
    prompt = prompt,  
    max_tokens=200,  
    temperature=0.750)

base_model = response.generations[0].text

print(base_model)

    - glaucoma  
    - primary open-angle glaucoma

The code uses the `cohere.Client()` method to input your API key. The above code also defines the prompt variable, which will contain instructions for the model.

The model’s objective for this experiment is to analyze a scientific paper’s abstract from the [PubMed website](https://pubmed.ncbi.nlm.nih.gov/) and identify a list of diseases. The cohere object’s `.generate()` method specifies the model type and provides the prompts and control parameters to achieve this.

The `max_tokens` parameter sets the limit for the number of new tokens the model can generate, and the temperature parameter controls the randomness level in the output.

The command model can identify diseases without examples or supplementary information.


## The Dataset

We will use the  [BC5CDR](https://paperswithcode.com/dataset/bc5cdr)  or BioCreative V Chemical Disease Relation Data. It comprises 1,500 manually annotated PubMed research papers, providing structured information on chemical-disease relations. The dataset is divided into training, validation, and testing sets containing 500 papers.

With this experiment, we aim to fine-tune a model that can identify and extract names of diseases/chemicals and their relationships from text. While research papers often describe relationships between chemicals and diseases in their abstracts, this information is typically unstructured. Manually finding “all chemicals influencing disease X” would require reading all papers mentioning “disease X.” Accurately extracting this structured information from unstructured text would facilitate more efficient searches.

Preprocess the dataset to adapt it for the Cohere service. It handles three file formats: CSV, JSONL, and plain text. We will use the JSONL format, which is consistent with the template below:

In [ ]:
{"prompt": "This is the first prompt",
"completion": "This is the first completion"}
{"prompt": "This is the second prompt",
"completion": "This is the second completion"}

💡The code is an example showing the extraction of disease names. Our final dataset will contain diseases, chemicals, and their corresponding relationships. We present a single step for extracting disease names to minimize the repetition of code. Please refer to [the notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2010%20-%20Create_Dataset_For_Cohere_Fine_Tuning.ipynb) at [towardsai.net/book](http://towardsai.net/book) for the complete preprocessing procedure and the resulting dataset.

Download the dataset in [JSON format](https://github.com/towardsai/rag-ebook-files/blob/main/bc5cdr.json) from [towardsai.net/book](http://towardsai.net/book) and open the JSON file using the code below. We also display a single row (passage) from the dataset to better illustrate the content and help understand the process. Each entry includes a text (which may be either a title or an abstract) and a list of entities that can be classified as either chemicals or diseases. For instance, in the example provided below, the first entity, Naloxone, is recognized as a chemical. The subsequent code will focus only on the information from the abstract, as the titles are short and provide limited details. (The printed output is simplified to improve understanding of the dataset and exclude non-essential information.)

In [ ]:
with open('bc5cdr.json') as json_file:
    data = json.load(json_file)

print(data[0])

    {'passages':  
    [{'document_id': '227508',  
    '**type**': 'title',  
    '**text**': 'Naloxone reverses the antihypertensive effect of clonidine.',  
    '**entities**': [  
    {'id': '0', 'text': ['Naloxone'], 'type': 'Chemical',},  
    {'id': '1', 'text': ['clonidine'], 'type': 'Chemical'}],  
    'relations': [...]},  
    {'document_id': '227508',  
    '**type**': 'abstract',  
    '**text**': 'In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) to 10(-5) M, did not influence stereoselective binding of [3H]-naloxone (8 nM), and naloxone, 10(-8) to 10(-4) M, did not influence clonidine-suppressible binding of [3H]-dihydroergocryptine (1 nM). These findings indicate that in spontaneously hypertensive rats the effects of central alpha-adrenoceptor stimulation involve activation of opiate receptors. As naloxone and clonidine do not appear to interact with the same receptor site, the observed functional antagonism suggests the release of an endogenous opiate by clonidine or alpha-methyldopa and the possible role of the opiate in the central control of sympathetic tone.',  
    '**entities**': [  
    {'id': '2', 'text': ['hypertensive'], 'type': 'Disease'},  
    {'id': '3', 'text': ['clonidine'], 'type': 'Chemical'},  
    {'id': '4', 'text': ['nalozone'], 'type': 'Chemical'},  
    {'id': '5', 'text': ['hypotensive'], 'type': 'Disease'},  
    {'id': '6', 'text': ['alpha-methyldopa'], 'type': 'Chemical'},  
    {'id': '7', 'text': ['naloxone'], 'type': 'Chemical'},  
    {'id': '8', 'text': ['Naloxone'], 'type': 'Chemical'},  
    {'id': '9', 'text': ['hypertensive'], 'type': 'Disease'},  
    {'id': '10', 'text': ['clonidine'], 'type': 'Chemical'},  
    {'id': '11', 'text': ['[3H]-naloxone'], 'type': 'Chemical'},  
    {'id': '12', 'text': ['naloxone'], 'type': 'Chemical'},  
    {'id': '13', 'text': ['clonidine'], 'type': 'Chemical'},  
    {'id': '14', 'text': ['[3H]-dihydroergocryptine'], 'type': 'Chemical'},  
    {'id': '15', 'text': ['hypertensive'], 'type': 'Disease'},  
    {'id': '16', 'text': ['naloxone'], 'type': 'Chemical',},  
    {'id': '17', 'text': ['clonidine'], 'type': 'Chemical'},  
    {'id': '18', 'text': ['clonidine'], 'type': 'Chemical'},  
    {'id': '19', 'text': ['alpha-methyldopa'], 'type': 'Chemical'}],  
    'relations': [...]}],  
    'dataset_type': 'train'}


We can loop through the dataset, extracting abstracts and related entities while including training instructions. There are two sets of instructions: the first helps the model understand the job, and the second shows how to construct the response:

In [ ]:
instruction = "The following article contains technical terms including diseases, drugs and chemicals. Create a list only of the diseases mentioned.\n\n"
output_instruction = "\n\nList of extracted diseases:\n"

The `instruction` variable sets the rules, and the `output_instruction` specifies the intended format for the output. Now, cycle through the dataset and format each instance:

In [ ]:
the_list = []
for item in data:
  chems = []
  dis = []

  if item['dataset_type'] == "test": continue;

  for ent in item['passages'][1]['entities']:
    if ent['type'] == "Chemical":
      if ent['text'][0] not in chems:
        chems.append( ent['text'][0] )

    if ent['type'] == "Disease":
      if ent['text'][0] not in dis:
        dis.append( ent['text'][0] )

Preparing each sample from the dataset requires iterating through all annotations and selecting only those related to diseases. This is necessary because the dataset also contains additional chemical labels. This will result in a dictionary with two keys: `prompt` and `completion`. The prompt key will consist of the paper abstract combined with specific instructions, and the `completion` key will list each disease name on a separate line.

This code will convert and save the dataset in JSONL format:

In [ ]:
# Writing to sample.json
with open("disease_instruct_all.jsonl", "w") as outfile:
  for item in the_list:
    outfile.write(json.dumps(item) + "\n")

The processed dataset is saved in a file named `disease_instruct_all.jsonl`. This file combines the training and validation sets to create 1,000 samples. The complete dataset comprises 3,000 samples, divided into three categories: 1,000 for diseases, 1,000 for chemicals, and 1,000 for their relationships.

💡The link to the final [preprocessed dataset](https://github.com/towardsai/rag-ebook-files/blob/main/both_rel_instruct_all.jsonl) is accessible at [towardsai.net/book](http://towardsai.net/book).